# Lab-2.3 Part 1: Basic FastAPI Service

## Objectives
- Learn FastAPI fundamentals
- Design API endpoints for LLM
- Implement request validation
- Manage model lifecycle

## Estimated Time: 60-90 minutes

---
## 1. FastAPI Basics

In [ ]:
# Check installations
try:
    import fastapi
    import uvicorn
    import pydantic
    print(f"✅ FastAPI: {fastapi.__version__}")
    print(f"✅ Uvicorn: {uvicorn.__version__}")
    print(f"✅ Pydantic: {pydantic.__version__}")
except ImportError as e:
    print(f"❌ Missing: {e}")
    print("\nInstall: pip install fastapi uvicorn[standard] pydantic")

### Create Basic API

Let's create a simple FastAPI application.

In [ ]:
%%writefile app_basic.py
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="LLM Service API", version="1.0.0")

class HealthResponse(BaseModel):
    status: str
    message: str

@app.get("/", response_model=HealthResponse)
async def root():
    return HealthResponse(
        status="ok",
        message="LLM Service is running"
    )

@app.get("/health")
async def health():
    return {"status": "healthy"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

### Run Server (in terminal)

```bash
# Start the server
uvicorn app_basic:app --host 0.0.0.0 --port 8000 --reload

# Server will be at: http://localhost:8000
# Interactive docs: http://localhost:8000/docs
```

In [ ]:
# Test API (if server is running)
import requests

API_URL = "http://localhost:8000"

try:
    response = requests.get(f"{API_URL}/health")
    print(f"✅ Server is running: {response.json()}")
except:
    print("❌ Server not running. Start with: uvicorn app_basic:app")

---
## 2. LLM Endpoints

In [ ]:
%%writefile app_llm.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import Optional, List
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

app = FastAPI(title="LLM API", version="1.0.0")

# Global model and tokenizer
model = None
tokenizer = None

class GenerateRequest(BaseModel):
    prompt: str = Field(..., min_length=1, max_length=2048)
    max_tokens: int = Field(default=100, ge=1, le=500)
    temperature: float = Field(default=0.8, ge=0.0, le=2.0)
    top_p: float = Field(default=0.95, ge=0.0, le=1.0)
    
    class Config:
        schema_extra = {
            "example": {
                "prompt": "Explain machine learning:",
                "max_tokens": 100,
                "temperature": 0.8,
                "top_p": 0.95
            }
        }

class GenerateResponse(BaseModel):
    text: str
    tokens_generated: int
    model: str

@app.on_event("startup")
async def startup_event():
    """Load model on startup."""
    global model, tokenizer
    
    print("Loading model...")
    model_name = "gpt2"  # Use small model for demo
    
    model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    print(f"✅ Model loaded: {model_name}")

@app.post("/v1/completions", response_model=GenerateResponse)
async def generate(request: GenerateRequest):
    """Generate text completion."""
    try:
        inputs = tokenizer(request.prompt, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=request.max_tokens,
                temperature=request.temperature,
                top_p=request.top_p,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        tokens_generated = len(outputs[0]) - len(inputs.input_ids[0])
        
        return GenerateResponse(
            text=generated_text,
            tokens_generated=tokens_generated,
            model="gpt2"
        )
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "model_loaded": model is not None
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

### Test the API

In [ ]:
# Test generate endpoint
import requests
import json

API_URL = "http://localhost:8000"

request_data = {
    "prompt": "The future of artificial intelligence is",
    "max_tokens": 50,
    "temperature": 0.8,
    "top_p": 0.95
}

print("Sending request...\n")

try:
    response = requests.post(
        f"{API_URL}/v1/completions",
        json=request_data,
        headers={"Content-Type": "application/json"},
    )
    
    result = response.json()
    
    print("Response:")
    print(json.dumps(result, indent=2))
    
except requests.exceptions.ConnectionError:
    print("❌ Server not running.")
    print("Start server: uvicorn app_llm:app --reload")

---
## 3. Chat Endpoint

In [ ]:
%%writefile app_chat.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Literal
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

app = FastAPI()

# Global state
model = None
tokenizer = None

class Message(BaseModel):
    role: Literal["system", "user", "assistant"]
    content: str

class ChatRequest(BaseModel):
    messages: List[Message]
    max_tokens: int = 100
    temperature: float = 0.8

class ChatResponse(BaseModel):
    message: Message
    tokens: int

@app.on_event("startup")
async def startup():
    global model, tokenizer
    model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Model loaded")

@app.post("/v1/chat/completions", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """Chat completions endpoint."""
    # Format messages into prompt
    prompt = ""
    for msg in request.messages:
        if msg.role == "system":
            prompt += f"System: {msg.content}\n"
        elif msg.role == "user":
            prompt += f"User: {msg.content}\n"
        elif msg.role == "assistant":
            prompt += f"Assistant: {msg.content}\n"
    
    prompt += "Assistant:"
    
    # Generate
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=request.max_tokens,
            temperature=request.temperature,
            do_sample=True,
        )
    
    response_text = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    tokens = len(outputs[0]) - len(inputs.input_ids[0])
    
    return ChatResponse(
        message=Message(role="assistant", content=response_text.strip()),
        tokens=tokens
    )

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8001)

### Test Chat Endpoint

In [ ]:
# Test chat endpoint
chat_request = {
    "messages": [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "What is machine learning?"}
    ],
    "max_tokens": 100,
    "temperature": 0.7
}

print("Testing chat endpoint...\n")

try:
    response = requests.post(
        "http://localhost:8001/v1/chat/completions",
        json=chat_request
    )
    
    result = response.json()
    print("Response:")
    print(json.dumps(result, indent=2))
    
except requests.exceptions.ConnectionError:
    print("❌ Server not running on port 8001")
    print("Start: uvicorn app_chat:app --port 8001")

---
## 4. Request Validation

In [ ]:
# Advanced validation example
from pydantic import BaseModel, Field, validator

class AdvancedGenerateRequest(BaseModel):
    prompt: str = Field(
        ...,
        min_length=1,
        max_length=4096,
        description="Input prompt for generation"
    )
    max_tokens: int = Field(
        default=100,
        ge=1,
        le=1000,
        description="Maximum tokens to generate"
    )
    temperature: float = Field(
        default=0.8,
        ge=0.0,
        le=2.0,
        description="Sampling temperature"
    )
    stop: Optional[List[str]] = Field(
        default=None,
        description="Stop sequences"
    )
    
    @validator('prompt')
    def validate_prompt(cls, v):
        """Custom validation for prompt."""
        # Check for forbidden patterns
        forbidden = ['<script>', 'DROP TABLE', 'rm -rf']
        for pattern in forbidden:
            if pattern.lower() in v.lower():
                raise ValueError(f"Forbidden pattern detected: {pattern}")
        return v
    
    @validator('temperature')
    def validate_temperature(cls, v, values):
        """Warn about extreme temperatures."""
        if v < 0.1 or v > 1.5:
            print(f"⚠️ Unusual temperature: {v}")
        return v

# Test validation
print("Testing request validation:\n")

# Valid request
try:
    valid_req = AdvancedGenerateRequest(
        prompt="Hello world",
        max_tokens=50
    )
    print(f"✅ Valid request: {valid_req.prompt}")
except Exception as e:
    print(f"❌ {e}")

# Invalid request (too many tokens)
try:
    invalid_req = AdvancedGenerateRequest(
        prompt="Test",
        max_tokens=2000  # Exceeds limit
    )
except Exception as e:
    print(f"\n❌ Invalid request caught: {e}")

# Forbidden pattern
try:
    forbidden_req = AdvancedGenerateRequest(
        prompt="<script>alert('test')</script>"
    )
except Exception as e:
    print(f"❌ Forbidden pattern caught: {e}")

---
## 5. Error Handling

In [ ]:
%%writefile app_error_handling.py
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic import BaseModel, ValidationError
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

class ErrorResponse(BaseModel):
    error: str
    detail: str
    request_id: str = None

@app.exception_handler(ValidationError)
async def validation_exception_handler(request, exc):
    """Handle validation errors."""
    logger.error(f"Validation error: {exc}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content=ErrorResponse(
            error="validation_error",
            detail=str(exc)
        ).dict()
    )

@app.exception_handler(Exception)
async def general_exception_handler(request, exc):
    """Handle general errors."""
    logger.error(f"Unexpected error: {exc}")
    return JSONResponse(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        content=ErrorResponse(
            error="internal_error",
            detail="An internal error occurred"
        ).dict()
    )

@app.post("/generate")
async def generate(prompt: str):
    # Simulate error for demo
    if "error" in prompt.lower():
        raise HTTPException(
            status_code=400,
            detail="Prompt contains 'error' keyword"
        )
    
    return {"text": f"Generated response for: {prompt}"}

---
## Summary

✅ **Completed**:
1. Created basic FastAPI application
2. Implemented LLM endpoints (/completions, /chat)
3. Added request validation with Pydantic
4. Implemented error handling
5. Managed model lifecycle

📚 **Key Concepts**:
- FastAPI automatic documentation
- Pydantic data validation
- Type hints and response models
- Startup events for model loading
- Exception handlers

➡️ **Next**: In `02-Async_Processing.ipynb`, we'll learn:
- Async/await patterns
- Concurrent request handling
- Streaming responses
- Request queuing

In [ ]:
print("✅ Lab 2.3 Part 1 Complete!")